In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))
    if len(filenames) > 3:
        print("...")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/captcha-hacker/sample_submission.csv
/kaggle/input/captcha-hacker/test/task1/x4LPcV5n6IvLj4vz.png
/kaggle/input/captcha-hacker/test/task1/W88fVMlAs5IpsXn4.png
/kaggle/input/captcha-hacker/test/task1/ZWDL6pUMfPu5c9jh.png
...
/kaggle/input/captcha-hacker/test/task2/jMalnsI5a5IWxYAi.png
/kaggle/input/captcha-hacker/test/task2/ihE9HHgyOINGEMcO.png
/kaggle/input/captcha-hacker/test/task2/ZATEVW3P5s0akZjd.png
...
/kaggle/input/captcha-hacker/test/task3/cXBlxYfvQWbiK7dn.png
/kaggle/input/captcha-hacker/test/task3/5gEp1jR9jNNfuqlk.png
/kaggle/input/captcha-hacker/test/task3/hEQ0WQtB9B7j8C2f.png
...
/kaggle/input/captcha-hacker/train/annotations.csv
/kaggle/input/captcha-hacker/train/task1/H85RQ6dbWUvLSIDV.png
/kaggle/input/captcha-hacker/train/task1/n2GC8uY1N4QfvVxe.png
/kaggle/input/captcha-hacker/train/task1/XOqfRx2R6SnoEjFr.png
...
/kaggle/input/captcha-hacker/train/task2/Mr4B2zxXk92hyzn9.png
/kaggle/input/captcha-hacker/train/task2/SIuRCnlK8VS91FhX.png
/kaggle/input/captcha-h

In [4]:
import csv
import cv2
import numpy as np
import random
import os

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms, utils, io
!pip install GPUtil
from GPUtil import showUtilization as gpu_usage
gpu_usage()         

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=786ecc8f0612cada1cc0c4e95d647af66bcc0149c6e9d9e62602f710f221afbc
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% |  0% |


In [5]:
TRAIN_PATH = "/kaggle/input/captcha-hacker/train"
TEST_PATH = "/kaggle/input/captcha-hacker/test"
# device = "cpu"
device = "cuda" 
# and change your settings/accelerator to GPU if you want it to run faster

In [6]:
class Task1Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task1")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
#         img = cv2.imread(f"{self.root}/{filename}")
        img = io.read_image(f"{self.root}/{filename}")
#         img = cv2.resize(img, (32, 3))
#         img = np.mean(img, axis=2)
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
            return torch.FloatTensor(img / 256), int(label)

    def __len__(self):
        return len(self.data)

In [7]:
# class Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(1024, 512),
#             nn.LeakyReLU(),
#             nn.Linear(512, 10)
#         )
        
        
#     def forward(self, x):
#         b, h, w = x.shape
#         x = x.view(b, h*w)
#         return self.layers(x)

class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()

        self.model = models.resnet18(pretrained=True)

        # 修改輸出層輸出數量
        self.model.fc = nn.Linear(512, 10)

    def forward(self, x):
#         b, h, w = x.shape
#         x = x.view(b, h*w)
# #         return self.model(x)
        logits = self.model(x)
        return logits

In [8]:
train_data = []
val_data = []

with open(f'{TRAIN_PATH}/annotations.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        if random.random() < 0.7:
            train_data.append(row)
        else:
            val_data.append(row)

train_ds = Task1Dataset(train_data, root=TRAIN_PATH)
train_dl = DataLoader(train_ds, batch_size=200, num_workers=4, drop_last=True, shuffle=True)

val_ds = Task1Dataset(val_data, root=TRAIN_PATH)
val_dl = DataLoader(val_ds, batch_size=200, num_workers=4, drop_last=False, shuffle=False)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [9]:
model = ResNet18().to(device)
torch.cuda.empty_cache()
# print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

image_change = transforms.Compose([transforms.Resize((384, 288)), transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

for epoch in range(20):
    print(f"Epoch [{epoch}]")
    model.train()
    for image, label in train_dl:
        image = image_change(image)
        image = image.to(device)
        label = label.to(device)
#         print("Shape of X [N, C, H, W]: ", image.shape)
        pred = model(image)
        loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del image, label
        
    sample_count = 0
    correct_count = 0
    model.eval()
    with torch.no_grad():
        for image, label in val_dl:
            image = image_change(image)
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_fn(pred, label)

            pred = torch.argmax(pred, dim=1)

            sample_count += len(image)
            correct_count += (label == pred).sum()
            
        del image, pred
        
    print("accuracy (validation):", correct_count / sample_count)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Epoch [0]
accuracy (validation): tensor(0.6752, device='cuda:0')
Epoch [1]
accuracy (validation): tensor(0.8222, device='cuda:0')
Epoch [2]
accuracy (validation): tensor(0.9726, device='cuda:0')
Epoch [3]
accuracy (validation): tensor(0.9863, device='cuda:0')
Epoch [4]
accuracy (validation): tensor(0.9983, device='cuda:0')
Epoch [5]
accuracy (validation): tensor(0.9983, device='cuda:0')
Epoch [6]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [7]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [8]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [9]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [10]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [11]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [12]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [13]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [14]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [15]
accuracy (validation): tensor(1., device='cud

In [10]:
torch.cuda.empty_cache()
test_data = []

with open(f'{TEST_PATH}/../sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)

test_ds = Task1Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=500, num_workers=4, drop_last=False, shuffle=False)


if os.path.exists('submission.csv'):
    csv_writer = csv.writer(open('submission.csv', 'a', newline=''))
else:
    csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
    csv_writer.writerow(["filename", "label"])


model.eval()
with torch.no_grad():
    for image, filenames in test_dl:
        image = image_change(image)
        image = image.to(device)

        pred = model(image)
        pred = torch.argmax(pred, dim=1)

        for i in range(len(filenames)):
            csv_writer.writerow([filenames[i], str(pred[i].item())])
            
    del image, pred

for filename, _ in test_data:
    if filename.startswith("task2") or filename.startswith("task3"):
        csv_writer.writerow([filename, 0])
    

